In [1]:
import cracknuts as cn

In [2]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [4]:
import random
import time


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(
        baudrate=cn.serial_enums.Baudrate.BAUDRATE_115200, 
        bytesize=cn.serial_enums.Bytesize.EIGHTBITS, 
        parity=cn.serial_enums.Parity.PARITY_NONE, 
        stopbits=cn.serial_enums.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


acq = cn.simple_glitch_acq(g1, init, do)

cn.show_panel(acq)

In [ ]:
cc = g1.get_current_config()


In [ ]:
cc.nut_voltage

In [ ]:
cc.nut_clock

In [ ]:
import struct

struct.unpack('>I', g1.register_read(base_address=0x43c10000, offset=0x1D0)[1])

In [ ]:
g1.register_read(base_address=0x43c10000, offset=0x150)[1].hex()

In [ ]:
struct.unpack('>I', g1.register_read(base_address=0x43c10000, offset=0x150)[1])

In [ ]:
cn.